# AdFlush Model
Test AdFlush ONNX, or train and test AdFlush upon H2O

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
# import matplotlib.pyplot as plt
# %matplotlib inline
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score
)
import time

Select the dataset to use during the session

In [40]:
# Available datasets
testset="../dataset/AdFlush_test.csv"
GAN_testset="../dataset/GAN_mutated_AdFlush.csv"
gnirts_testset="../dataset/JS_obfuscated_gnirts.csv"
javascript_obfuscator_testset="../dataset/JS_obfuscated_javascript_obfuscator.csv"
wobfuscator_testset="../dataset/JS_obfuscated_wobfuscator.csv"

# Select dataset to import
DATASET=gnirts_testset
ISMUTATED=False
if DATASET==testset or DATASET==GAN_testset:
    data_df=pd.read_csv(DATASET, index_col=0)
elif DATASET==gnirts_testset or DATASET==javascript_obfuscator_testset or DATASET==wobfuscator_testset:
    js_features=['brackettodot', 'num_get_storage', 'num_set_storage',
        'num_get_cookie', 'num_requests_sent', 'ng_0_0_2', 'ng_0_15_15', 'ng_2_13_2',
        'ng_15_0_3', 'ng_15_0_15', 'ng_15_15_15', 'avg_ident',
        'avg_charperline']
    test_df=pd.read_csv(testset,index_col=0)
    mutate_df=pd.read_csv(DATASET,index_col=0)
    mutate_df=mutate_df[js_features].copy()
    mutate_performed=mutate_df.index
    test_df=test_df.loc[mutate_performed].copy()
    test_df.loc[mutate_performed,js_features]=mutate_df.loc[mutate_performed,js_features]
    data_df=test_df.copy()
    ISMUTATED=True
data_df

,content_policy_type,url_length,brackettodot,is_third_party,keyword_char_present,num_get_storage,num_set_storage,num_get_cookie,num_requests_sent,req_url_33,...,fqdn_27,ng_0_0_2,ng_0_15_15,ng_2_13_2,ng_15_0_3,ng_15_0_15,ng_15_15_15,avg_ident,avg_charperline,label
273066,0.742155,51.0,2.859708,1.0,0.0,0.0,0.0,0.0,0.0,0.248519,...,-0.967004,0.020967,0.042119,0.0,0.070443,0.043774,0.022991,0.160803,19420.0,1.0
76875,0.474811,70.0,2.608696,1.0,1.0,0.0,0.0,0.0,0.0,0.112937,...,-0.845820,0.028986,0.050725,0.0,0.050725,0.086957,0.065217,0.230415,440.0,1.0
642137,0.036026,22.0,2.860262,0.0,0.0,0.0,0.0,0.0,0.0,0.323858,...,-0.524810,0.020743,0.041889,0.0,0.068053,0.046145,0.025895,0.180937,16927.0,0.0
564095,0.474811,43.0,2.821429,0.0,0.0,0.0,0.0,0.0,0.0,0.329914,...,-0.924073,0.003312,0.006889,0.0,0.010664,0.007882,0.004637,0.171657,69549.0,0.0
665496,0.474811,30.0,2.831615,0.0,0.0,0.0,0.0,0.0,0.0,0.326959,...,-1.248899,0.021672,0.044582,0.0,0.074923,0.051393,0.030341,0.167495,5626.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13740,0.036026,28.0,2.851948,0.0,0.0,0.0,0.0,0.0,0.0,0.198387,...,-1.193333,0.022518,0.045694,0.0,0.074951,0.051611,0.029750,0.173748,7494.0,0.0
711321,0.474811,62.0,2.859813,1.0,0.0,0.0,0.0,0.0,0.0,0.104110,...,-0.828099,0.022165,0.045013,0.0,0.078431,0.047570,0.025575,0.159865,20444.0,1.0
164567,0.474811,50.0,2.877346,0.0,0.0,0.0,0.0,0.0,0.0,0.135737,...,-0.487274,0.023319,0.046687,0.0,0.085368,0.047138,0.023869,0.154570,70417.0,0.0
665529,0.384715,50.0,2.816143,0.0,0.0,0.0,0.0,0.0,0.0,0.052651,...,-0.904423,0.020331,0.042169,0.0,0.068524,0.050452,0.030873,0.171883,4618.0,0.0


In [41]:
def metrics(true, pred, _is_mutated):
    print(f"Accuracy : {accuracy_score(true, pred)} ")
    print(f"Precision : {precision_score(true, pred)} ")
    print(f"Recall : {recall_score(true, pred)} ")
    print(f"F1 : {f1_score(true, pred)} ")
    # Number of attacks
    total_attacks = len(true)
    
    # Number of successful attacks (misclassifications)
    successful_attacks = sum(true != pred)
    tn, fp, fn, tp = confusion_matrix(true, pred).ravel()

    # Calculate FNR
    fnr = fn / (tp + fn)
    print('False Negative Rate:', fnr)

    # Calculate FPR
    fpr = fp / (fp + tn)
    print('False Positive Rate:', fpr)
    # ASR
    if _is_mutated:
        asr = successful_attacks / total_attacks
        print("Attack Success Rate: ",asr)

### 1. AdFlush_ONNX

In [42]:
import onnx
import onnxruntime as ort

In [43]:
print('Loading Model')
model = onnx.load('../model/AdFlush.onnx')
# Check that the IR is well formed
try:
    print("Checking model integrity")
    onnx.checker.check_model(model)
    # Create an ONNX runtime session
    ort_session = ort.InferenceSession('../model/AdFlush.onnx')
    input_data = data_df.drop('label', axis=1).values.astype('float32')
    label=data_df['label'].astype(int)
    input_name = ort_session.get_inputs()[0].name
    label_name = ort_session.get_outputs()[0].name

    # Run the inference session to get the prediction results
    print('Running Inference Session')
    start_time=time.time()
    pred = ort_session.run([label_name], {input_name: input_data})[0]
    print("Inference time elapsed: ", time.time()-start_time, "for ", len(label), " samples.")
    metrics(label.astype(int),pred.astype(int),ISMUTATED)
    roc_auc_score(label, pred.astype('float64'))    
    
except Exception as e:
    print("Error in loading model: ",e)

Loading Model
Running Inference Session
Inference time elapsed:  0.0670006275177002 for  15386  samples.
Accuracy : 0.9249967502924736 
Precision : 0.987251828631139 
Recall : 0.8121024583118446 
F1 : 0.891152612714582 
False Negative Rate: 0.1878975416881554
False Positive Rate: 0.006374751802696207
Attack Success Rate:  0.07500324970752632


### 2. AdFlush_H2O_MOJO

In [52]:
import h2o
from h2o.automl import H2OAutoML
h2o.init(nthreads = 12, max_mem_size = "64g", enable_assertions = False)
h2o.no_progress()
custom_path=""

Checking whether there is an H2O instance running at http://localhost:54321. connected.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,4 mins 16 secs
H2O_cluster_timezone:,Asia/Seoul
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.1
H2O_cluster_version_age:,4 months and 4 days
H2O_cluster_name:,H2O_from_python_chaejin_lim_xy39qc
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,63.90 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


#### 2-1. Training AdFlush with H2O AutoML (optional)

In [53]:
train=pd.read_csv('../dataset/AdFlush_train.csv',index_col=0)
test=data_df.copy() #Use selected Dataset to test
train.reset_index(inplace=True, drop=True)
test.reset_index(inplace=True, drop=True)

y = 'label'
x = list(train)
x.remove(y)

h2o_train = h2o.H2OFrame(train)
h2o_test = h2o.H2OFrame(test)

h2o_train[y] = h2o_train[y].asfactor()
h2o_test[y] = h2o_test[y].asfactor()

# Select runtime for exploration
MAXRUNTIME=3600
aml = H2OAutoML(max_runtime_secs=MAXRUNTIME, max_models=None, exclude_algos=['XGBoost', 'StackedEnsemble'], nfolds=5)

aml.train(x = x, y = y, training_frame = h2o_train, leaderboard_frame = h2o_test)
custom_path=aml.leader.save_mojo('../model/AdFlush_custom_mojo')

Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_1_AutoML_2_20231025_144745


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    2                  2                           32783                  15           15           15            1060          1539          1299.5

ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.1784367063861446
RMSE: 0.42241769184794403
LogLoss: 0.5450574544259663
Mean Per-Class Error: 0.07509788983251584
AUC: 0.97624922197045
AUCPR: 0.9713151401486603
Gini: 0.9524984439409001

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4164233654643016
       0       1       Error    Rate
-----  ------  ------  -------  ------------------
0      380957  18055   0.0452   (18055.0/399012.0)
1      27823   237293  0.1049   (27823.0/265116.0)
Total  408780  255348  0.0691   (45878.0/664128.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.416423     0.911852  189
max f2                       0.371069     0.923647  263
max f0point5                 0.449678     0.933197  137
max accuracy                 0.416423     0.93092   189
max precision                0.518201     1         0
max recall                   0.324047     1         398
max specificity              0.518201     1         0
max absolute_mcc             0.416423     0.855498  189
max min_per_class_accuracy   0.397002     0.92512   222
max mean_per_class_accuracy  0.40589      0.926534  208
max tns                      0.518201     399012    0
max fns                      0.518201     264986    0
max fps                      0.323148     399012    399
max tps                      0.324047     265116    398
max tnr                      0.518201     1         0
max fnr                      0.518201     0.99951   0
max fpr                      0.323148     1         399
max tpr                      0.324047     1         398

Gains/Lift Table: Avg response rate: 39.92 %, avg score: 40.03 %
group    cumulative_data_fraction    lower_threshold    lift    cumulative_lift    response_rate    score     cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain    cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ------  -----------------  ---------------  --------  --------------------------  ------------------  --------------  -------------------------  ------  -----------------  --------------------
1        1                           0                  1       1                  0.399194         0.400313  0.399194                    0.400313            1               1                          0       0                  0

ModelMetricsBinomial: gbm
** Reported on cross-validation data. **

MSE: 0.17100272664889068
RMSE: 0.41352475941458533
LogLoss: 0.5290215471908971
Mean Per-Class Error: 0.07583162902297612
AUC: 0.9706303342273642
AUCPR: 0.9635425545315798
Gini: 0.9412606684547284

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4144892060678761
       0       1       Error    Rate
-----  ------  ------  -------  ------------------
0      377849  21163   0.053    (21163.0/399012.0)
1      26147   238969  0.0986   (26147.0/265116.0)
Total  403996  260132  0.0712   (47310.0/664128.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.414489     0.909928  197
max f2                       0.37108      0.921384

In [57]:
path = '../model/AdFlush_mojo.zip'
if custom_path!="":
    path=custom_path    
h2o_model = h2o.import_mojo(path)

pred = h2o_model.predict(h2o_test)
pred = pred.as_data_frame().predict.to_list()
metrics(test['label'], pred, ISMUTATED)
if ISMUTATED:
    roc_auc_score(test['label'], pred)

Accuracy : 0.8666320031197192 
Precision : 0.831367716951241 
Recall : 0.8119305483926422 
F1 : 0.8215341798573665 
False Negative Rate: 0.18806945160735775
False Positive Rate: 0.10011495454070436
Attack Success Rate:  0.13336799688028078


0.8559077969259689

In [ ]:
##### MOJO zip model to ONNX Converter

# path = 'AdFlush_mojo.zip'
# onnx_model = onnxmltools.convert_h2o(path, target_opset=9)
# input_name = onnx_model.graph.input[0].name
# input_name
# # onnx_model = order_conversion(
# #     onnx_graph=onnx_model,
# #     input_op_names_and_order_dims={f"{input_name}": ['None', 'None']},
# #     non_verbose=True
# # )
# onnxmltools.utils.save_model(onnx_model, 'AdFlush.onnx')


In [ ]:
h2o_model

In [ ]:
h2o_model.shap_summary_plot(h2o_test)

In [ ]:
#  'response_column': 'label',
#  'ignored_columns': [],
#  'ignore_const_cols': True,
#  'offset_column': None,
#  'weights_column': None,
#  'balance_classes': False,
#  'class_sampling_factors': None,
#  'max_after_balance_size': 5.0,
#  'max_confusion_matrix_size': 20,
#  'ntrees': 166,
#  'max_depth': 16,
#  'min_rows': 5.0,
#  'nbins': 20,
#  'nbins_top_level': 1024,
#  'nbins_cats': 1024,
#  'r2_stopping': 1.7976931348623157e+308,
#  'stopping_rounds': 0,
#  'stopping_metric': 'logloss',
#  'stopping_tolerance': 0.0012270834667786677,
#  'max_runtime_secs': 778.0,
#  'seed': 7352808894752235504,
#  'build_tree_one_node': False,
#  'learn_rate': 0.1,
#  'learn_rate_annealing': 1.0,
#  'distribution': 'bernoulli',
#  'quantile_alpha': 0.5,
#  'tweedie_power': 1.5,
#  'huber_alpha': 0.9,
#  'checkpoint': None,
#  'sample_rate': 0.9,
#  'sample_rate_per_class': None,
#  'col_sample_rate': 0.7,
#  'col_sample_rate_change_per_level': 1.0,
#  'col_sample_rate_per_tree': 0.4,
#  'min_split_improvement': 1e-05,
#  'histogram_type': 'UniformAdaptive',
#  'max_abs_leafnode_pred': 1.7976931348623157e+308,
#  'pred_noise_bandwidth': 0.0,
#  'categorical_encoding': 'Enum',
#  'calibrate_model': False,
#  'calibration_frame': None,
#  'calibration_method': 'PlattScaling',
#  'custom_metric_func': None,
#  'custom_distribution_func': None,
#  'export_checkpoints_dir': None,
#  'in_training_checkpoints_dir': None,
#  'in_training_checkpoints_tree_interval': 1,
#  'monotone_constraints': None,
#  'check_constant_response': True,
#  'gainslift_bins': -1,
#  'auc_type': 'AUTO',
#  'interaction_constraints': None,
#  'auto_rebalance': True}